In [5]:
import os
import cv2
import numpy as np
import networkx as nx
import torch
import torchvision.transforms as transforms
from skimage.segmentation import felzenszwalb
from skimage.feature.texture import greycomatrix, greycoprops
from skimage.color import rgb2gray
from tqdm import tqdm

ImportError: cannot import name 'greycomatrix' from 'skimage.feature.texture' (C:\Users\EDWIN\AppData\Roaming\Python\Python313\site-packages\skimage\feature\texture.py)

In [ ]:
# Define dataset path
DATASET_PATH = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\Dataset" 
CATEGORIES = ["Negative", "Positive"]

# Image preprocessing
IMG_SIZE = 256  # Resize images

# Function to extract texture features using GLCM (Gray-Level Co-occurrence Matrix)
def extract_texture_features(gray_image):
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = greycomatrix(gray_image, distances=distances, angles=angles, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast').mean()
    dissimilarity = greycoprops(glcm, 'dissimilarity').mean()
    homogeneity = greycoprops(glcm, 'homogeneity').mean()
    energy = greycoprops(glcm, 'energy').mean()
    correlation = greycoprops(glcm, 'correlation').mean()
    return [contrast, dissimilarity, homogeneity, energy, correlation]

# Function to process image and create a graph
def image_to_graph(image_path):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    # Apply graph-based segmentation
    segments = felzenszwalb(image, scale=100, sigma=0.5, min_size=50)

    # Convert image to grayscale for texture extraction
    gray_image = rgb2gray(image)

    # Create a graph
    G = nx.Graph()

    # Get unique segments (superpixels)
    unique_segments = np.unique(segments)

    for seg_id in unique_segments:
        # Get segment mask
        mask = segments == seg_id
        y, x = np.where(mask)

        if len(y) == 0:  # Skip empty segments
            continue

        # Compute node features (color & texture)
        mean_color = np.mean(image[y, x], axis=0)  # RGB mean color
        texture_features = extract_texture_features(gray_image)  # Texture properties

        # Add node to the graph
        G.add_node(seg_id, color=mean_color.tolist(), texture=texture_features)

    # Add edges based on adjacency of segments
    for y in range(segments.shape[0] - 1):
        for x in range(segments.shape[1] - 1):
            seg1 = segments[y, x]
            seg2 = segments[y + 1, x]
            seg3 = segments[y, x + 1]

            if seg1 != seg2:
                G.add_edge(seg1, seg2)

            if seg1 != seg3:
                G.add_edge(seg1, seg3)

    return G



ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Process dataset and save graphs
graphs = []
labels = []
for category in CATEGORIES:
    label = 1 if category == "Positive" else 0
    folder_path = os.path.join(DATASET_PATH, category)
    
    print(f"Processing {category} images...")
    for img_name in tqdm(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        graph = image_to_graph(img_path)
        graphs.append(graph)
        labels.append(label)

# Save dataset (can be used with PyTorch Geometric)
torch.save({"graphs": graphs, "labels": labels}, "graph_dataset.pt")

print("Graph dataset saved successfully!")
